In [2]:
from bs4 import BeautifulSoup
import requests
import html5lib
import re
import pandas as pd
import numpy as np
# pip install pyopenssl ndg-httpsclient pyasn1

In [3]:
cookies = {
    'ccsid': '668-8707266-8901325',
    '__qca': 'P0-1002348396-1661184067604',
    'p': 'ZqHh-ArV88dYkVHfgapFDgVJRXU5lLzNc4VfzdNPfI5ZMVgd',
    'likely_has_account': 'true',
    'srb_8': '0_wl',
    'locale': 'en',
    'u': 'oTJu21ixofG3HvrbsDKN1hg8e8FfKW_FUT1R8ZnFKULdyBce',
    'csm-sid': '696-1026276-2871101',
    'logged_out_browsing_page_count': '2',
}

headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Accept-Language': 'en-US,en;q=0.9,id;q=0.8',
    'Cache-Control': 'max-age=0',
    'Connection': 'keep-alive',
    # 'Cookie': 'ccsid=668-8707266-8901325; __qca=P0-1002348396-1661184067604; p=ZqHh-ArV88dYkVHfgapFDgVJRXU5lLzNc4VfzdNPfI5ZMVgd; likely_has_account=true; srb_8=0_wl; locale=en; u=oTJu21ixofG3HvrbsDKN1hg8e8FfKW_FUT1R8ZnFKULdyBce; csm-sid=696-1026276-2871101; logged_out_browsing_page_count=2',
    'If-None-Match': 'W/"4279d1cc80c6f7b6843a18660d55f9f2"',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'none',
    'Sec-Fetch-User': '?1',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
    'sec-ch-ua': '"Not?A_Brand";v="8", "Chromium";v="108", "Google Chrome";v="108"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
}


In [4]:
cookies2 = {
    'ccsid': '668-8707266-8901325',
    '__qca': 'P0-1002348396-1661184067604',
    'p': 'ZqHh-ArV88dYkVHfgapFDgVJRXU5lLzNc4VfzdNPfI5ZMVgd',
    'likely_has_account': 'true',
    'srb_8': '0_wl',
    'locale': 'en',
    'u': 'oTJu21ixofG3HvrbsDKN1hg8e8FfKW_FUT1R8ZnFKULdyBce',
    'csm-sid': '696-1026276-2871101',
    'logged_out_browsing_page_count': '2',
    '_session_id2': 'ae3b31e88be4ac3d2df2cc4e36100b58',
}

headers2 = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Accept-Language': 'en-US,en;q=0.9,id;q=0.8',
    'Cache-Control': 'max-age=0',
    'Connection': 'keep-alive',
    # 'Cookie': 'ccsid=668-8707266-8901325; __qca=P0-1002348396-1661184067604; p=ZqHh-ArV88dYkVHfgapFDgVJRXU5lLzNc4VfzdNPfI5ZMVgd; likely_has_account=true; srb_8=0_wl; locale=en; u=oTJu21ixofG3HvrbsDKN1hg8e8FfKW_FUT1R8ZnFKULdyBce; csm-sid=696-1026276-2871101; logged_out_browsing_page_count=2; _session_id2=ae3b31e88be4ac3d2df2cc4e36100b58',
    'If-None-Match': 'W/"b850254e01551ab34564b673d5620452"',
    'Referer': 'https://www.goodreads.com/shelf/show/to-read?page=1',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-User': '?1',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
    'sec-ch-ua': '"Not?A_Brand";v="8", "Chromium";v="108", "Google Chrome";v="108"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
}

In [17]:
def clean_publish_year(raw_year):
    if type(raw_year) is None:
        return "Missing"
    result = re.search(r"(\b\d+)$", raw_year)
    if result == None:
        return 0
    else:
        return result.group(1)

def clean_rating(raw_rating):
    if type(raw_rating) is None:
        return 0
    result = re.search(r"(\b\d+.\b\d+)", raw_rating)
    if result == None:
        return 0
    else:
        return result.group(1)

def clean_raters(raw_raters):
    if type(raw_raters) is None:
        return 0
    result = re.search(r"((?:\d+,?\d+)*) ratings", raw_raters)
    if result == None:
        return 0
    else:
        return result.group(1)
    
def clean_desc(raw_desc):
    spanList = []
    desc = ""
    if raw_desc == None:
        return "Missing"
    else:
        spanList = raw_desc.find_all("span")
        if len(spanList)== 2:
            desc = spanList[1].text
        else:
            desc = spanList[0].text

        return desc
# def clean_readers(raw_readers):
#     result = re.search(r"(\b\d+)", raw_readers)
#     if result == None:
#         return "None"
#     else:
#         return result.group(1)
# <span id="freeTextContainer1584189417832634179">

def solve_nonetype_error(value):
    if value is not None:
        return value.text.strip()
    else:
        return "Missing"



In [20]:

books = []
for page in range(1, 26):
    response = requests.get("https://www.goodreads.com/shelf/show/horror?page=" + str(page), cookies = cookies, headers=headers)
    html_soup = BeautifulSoup(response.text, 'html5lib')
    book_container = html_soup.find_all('div', class_ = "elementList")
    print("this is page " , page)
    for book in book_container:
        link = book.a['href']
        completeLink = "https://www.goodreads.com" + link
        # getGenre = requests.get(completeLink)
        getDesc = requests.get(completeLink,  cookies = cookies2, headers=headers2)
        descParsing = BeautifulSoup(getDesc.text, 'html5lib')
        desc = descParsing.find("div", attrs= {"id" : "description", "class" : "readable stacked", "style" : "right:0"})
        desc_text = clean_desc(desc)
        book_data = [desc_text]
        books.append(desc_text)
        
booksDesc = pd.DataFrame(books, columns=["Description"])
booksDesc.head()

# <span id="freeTextContainer1584189417832634179">

this is page  1
this is page  2
this is page  3
this is page  4
this is page  5
this is page  6
this is page  7
this is page  8
this is page  9
this is page  10
this is page  11
this is page  12
this is page  13
this is page  14
this is page  15
this is page  16
this is page  17
this is page  18
this is page  19
this is page  20
this is page  21
this is page  22
this is page  23
this is page  24
this is page  25


,Description
0,Jack Torrance's new job at the Overlook Hotel ...
1,"Welcome to Derry, Maine ...It’s a small city, ..."
2,You can find an alternative cover edition for ...
3,"A modern classic, Carrie introduced a distinct..."
4,'This is an alternate Cover Edition for ASIN: ...


In [50]:
booksDesc.to_csv("HorrorOnlyDesc.csv")


In [74]:
df = pd.read_csv("HorrorOnlyDesc.csv")

df2 = pd.read_csv("HorrorGenreCleaned.csv")

df["Description"] = df["Description"].astype(str)
joined = pd.merge(df2, df)
joined = joined.drop("Unnamed: 0", axis = 1)


joined.head()

joined.to_csv("HorrorCleanedAndJoined.csv")

In [54]:
df = pd.read_csv("HorrorCleanedAndJoined.csv")
df

,Unnamed: 0.1,Unnamed: 0,Description
0,0,0,Jack Torrance's new job at the Overlook Hotel ...
1,1,1,"Welcome to Derry, Maine ...It’s a small city, ..."
2,2,2,You can find an alternative cover edition for ...
3,3,3,"A modern classic, Carrie introduced a distinct..."
4,4,4,'This is an alternate Cover Edition for ASIN: ...
...,...,...,...
1294,1294,1294,A woman of snow ... a midnight caller keeping ...
1295,1295,1295,"In a tale of ancient evil, Bram Stoker creates..."
1296,1296,1296,\n Get Out\n meets \n The Stepford Wives\n i...
1297,1297,1297,Missing
